# Импорты

In [ ]:
import re

import pandas as pd
from matplotlib import pyplot as plt

# Для более качественных графиков
%config InlineBackend.figure_format='retina'
plt.rcParams["figure.dpi"] = 150

# Обогащение, разметка датасетов

## Регулярные выражения

Регулярные выражение - формальный язык для составления текстовых шаблонов. При помощи этих шаблонов можно искать и обрабатывать текст

[Документация](https://docs.python.org/3/library/re.html)

__Специальные символы регулярных выражений__

Внутри регулярного выражения символы `.   ^   $  *  +  ?   {  } [  ]  \  |  ( ) -` имеют специальное значение. 
* Если эти символы требуется выводить как есть, то их следует экранировать с помощью слэша `\`.
* Некоторые сnециальные символы теряют свое особое значение, если их разместить внутри квадратных скобок `[]`. В этом случае экранировать их не нужно. 

__Основные методы__

* `re.match()` - Этот метод ищет по заданному шаблону в начале строки. Возвращает первое вхождение подстроки в виде объекта SRE_Match object, из которого:
    * можно получить результирующую подстроку с помощью функции group
    * индексы начальной и конечной позиции с помощью функций start() и end(), соответственно.
* `re.search()` - ищет по заданному шаблону во всей строке
* `re.findall()` - возвращает список всех найденных совпадений (подстрок).
* `re.split()` - разделяет строку по заданному шаблону
* `re.sub()` - заменяет шаблон на указанную подстроку.

В квадратных скобках `[]` можно указать символы, которые могут встречаться на этом месте в строке. Можно перечислять символы подряд или указать их диапазон через тире. Например:
* `[09]` - соответствует цифре 0 или 9 <br>
* `[0-9]` - соответствует одной цифре от 0 до 9 <br>
* `[абв]` - соответствует букве "а", "б" или "в" <br>
* `[а-г]` - соответствует букве "а", "б", "в" или "г" <br>
* `[а-я]` - соответствует любой букве от "а" до "я", кроме буквы "ё" (т.к. "ё" находится вне непрерывного дипаозона символов русского алфавита) <br>
* `[а-яё]` - соответствует любой букве от "а" до "я" <br>
* `[АБВ]` - соответствует букве "А", "Б" или "В" <br>
* `[А-ЯЁ]` - соответствует любой букве от "А" до "Я" <br>
* `[а-яА-ЯёЁ]` - соответствует любой русской букве в любом регистре <br>
* `[0-9а-яА-ЯёЁа-zА-Z]` - любая цифра и любая буква независимо от регистра и языка

Вместо перечисления символов можно использовать стандартные классы: <br>

* . - любой символ, кроме перевода строки (если точка не экранирована и не заключена в квадратные скобки)
* \d - соответствует любой цифре (эквивалентно [0-9]) <br>
* \w - соответствует любой букве, цифре или символу подчеркивания ([a-zA-Zа-яЁА-ЯЁ0-9_]) <br>
* \s - любой пробельный символ (пробел, перевод строки, табуляция и т.д.) <br>
* \D - не цифра (эквивалентно [^0-9]) <br>
* \W - не буква, не цифра и не символ подчеркивания (эквивалентно [^a-zA-Zа-яЁА-ЯЁ0-9_]) <br>
* \S - не пробельный символ <br>
* \b - обозначение левой или правой границы слова (где слово трактуется как последовательность букв или цифр)

С помощью квантификаторов задается количество вхождений символа в строку. Указывается после символа, к которому относится разрешенное количество повторений: <br>

* `{n}` - n вхождений символа в строку. Например. шаблон `r"[0-9]{2}"` соответствует двум вхождениям любой цифры
* `{n,}` - n или более вхождений символа в строку. Например. шаблон `r"[0-9]{2,}"` соответствует двум и более вхождениям любой цифры
* `{n,m}` - не менее n и не более m вхождений символа в строку. Числа указываются через запятую без пробела. 
    * Например, шаблон `r"[0-9]{2,4}"` соответствует от двух до четырех вхождениям любой цифры
* `*` - ноль или большее число вхождений символа в строку. Эквивалентно комбинации `{0,}`
* `+` - одно или большее число вхождений символа в строку. Эквивалентно комбинации `{1,}`
* `?` - ни одного или одно вхождение символа в строку. Эквивалентно комбинации `{0,1}`. 

Еще полезные символы:

* `^` - привязка к началу строки или подстроки.
* `$` - привязка к концу строки или подстроки.

In [ ]:
df = pd.read_csv("data/train.csv")

default_df = df[df["дефолт"] == 1]
print(*list(df["речь"].iloc[:5]), sep="\n")

In [ ]:
re.findall(r"(solid|medium)", "guy with a solid job and a medium income")

In [ ]:
print(*list(default_df["речь"].iloc[5:10]), sep="\n")

In [ ]:
re.findall(r"(?<!not\s)sure\d", "sure1 not sure2")

Задание: используя одно из регулярных выражений, создайте новый столбец в таблице

# Кросс-валидация

In [ ]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

model = LogisticRegression()

X = df.drop(columns="дефолт")
y = df["дефолт"]
num_cols = X.select_dtypes(exclude=object).columns
X = X[num_cols]
X = X.fillna(0)
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

metrics = []

kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X_train):
    cur_X_train = X_train[train_index]
    cur_y_train = y_train.iloc[train_index]
    cur_X_val = X_train[test_index]
    cur_y_val = y_train.iloc[test_index]

    model.fit(X_train, y_train)
    pred = model.predict(cur_X_val)
    metrics.append(f1_score(y_true=cur_y_val, y_pred=pred))

In [ ]:
metrics